# Rigorous Thermodynamic Decomposition of Urea's Effects on the Solubility of Polyethylene Glycol
Stefan Hervø-Hansen<sup>a,</sup> and Nobuyuki Matubayasi<sup>a,</sup><br><br>
<sup>a</sup> Division of Chemical Engineering, Graduate School of Engineering Science, Osaka University, Toyonaka, Osaka 560-8531, Japan.<br>
<sup></sup> Correspondence may be addressed to: stefan@cheng.es.osaka-u.ac.jp and nobuyuki@cheng.es.osaka-u.ac.jp.

## Part 2: ERmod

### Introduction
Here we aim to provide a detailed thermodynamic analysis of how urea influence the solvation of polyethylene glycol (PEG). By utilizing molecular dynamics simulations, we can gain atomic insight into the mechanism which underpins the change in excess chemical potential of PEG with the addition of urea. Understanding these effects is crucial for applications in biochemistry and materials science, where PEG and its derivatives are widely used. The following sections detail the methods and materials employed in our simulations and analyses.

### Methods & Materials
Molecular dynamics simulations were conducted using the OpenMM (8.0)[<sup>1</sup>](#fn1) software package. The details of these simulations can be found in the [Part 1 Jupyter notebook](Simulations.ipynb). For the simulation of PEG, a CHARMM-derived force field (C35r) was utilized, which has previously been shown to reproduce the hydrodynamic radii and shape anisotropy of PEG[<sup>2</sup>](#fn2). The PEG force field was combined with the SPC/E force field for water[<sup>3</sup>](#fn3) and a Kirkwood-Buff derived force field for urea[<sup>4</sup>](#fn4).

The isothermal-isobaric ensemble was sampled using a combination of a "Middle" discretization Langevin leap-frog integrator[<sup>5,</sup>](#fn5)[<sup>6</sup>](#fn6) and a Monte Carlo barostat[<sup>7,</sup>](#fn7)[<sup>8</sup>](#fn8). The trajectories were analyzed using MDTraj[<sup>9</sup>](#fn9) for structural properties, while ERmod[<sup>10</sup>](#fn10) was used for the calculation of solvation free energies. The calculation of solvation free energy can be found in the [Part 2 Jupyter notebook](ERmod.ipynb) and the analysis of data can be found in [Part 3 Jupyter notebook](Analysis.ipynb)

### References
1. <span id="fn1"> P. Eastman, et al., OpenMM 8: Molecular Dynamics Simulation with Machine Learning Potentials. J. Phys. Chem. B 128, 109–116 (2023). </span><br>
2. <span id="fn2"> H. Lee, R. M. Venable, A. D. MacKerell Jr., R. W. Pastor, Molecular Dynamics Studies of Polyethylene Oxide and Polyethylene Glycol: Hydrodynamic Radius and Shape Anisotropy. Biophysical Journal 95, 1590–1599 (2008). </span><br>
3. <span id="fn3"> H. J. C. Berendsen, J. R. Grigera, T. P. Straatsma, The missing term in effective pair potentials. J. Phys. Chem. 91, 6269–6271 (1987). </span><br>
4. <span id="fn4"> S. Weerasinghe, P. E. Smith, A Kirkwood−Buff Derived Force Field for Mixtures of Urea and Water. J. Phys. Chem. B 107, 3891–3898 (2003). </span><br>
5. <span id="fn5"> B. Leimkuhler, C. Matthews, Efficient molecular dynamics using geodesic integration and solvent–solute splitting. Proc. R. Soc. A. 472, 20160138 (2016). </span><br>
6. <span id="fn6"> Z. Zhang, X. Liu, K. Yan, M. E. Tuckerman, J. Liu, Unified Efficient Thermostat Scheme for the Canonical Ensemble with Holonomic or Isokinetic Constraints via Molecular Dynamics. J. Phys. Chem. A 123, 6056–6079 (2019). </span><br>
7. <span id="fn7"> K.-H. Chow, D. M. Ferguson, Isothermal-isobaric molecular dynamics simulations with Monte Carlo volume sampling. Computer Physics Communications 91, 283–289 (1995). </span><br>
8. <span id="fn8"> J. Åqvist, P. Wennerström, M. Nervall, S. Bjelic, B. O. Brandsdal, Molecular dynamics simulations of water and biomolecules with a Monte Carlo constant pressure algorithm. Chemical Physics Letters 384, 288–294 (2004). </span><br>
9. <span id="fn9"> R. T. McGibbon, et al., MDTraj: A Modern Open Library for the Analysis of Molecular Dynamics Trajectories. Biophysical Journal 109, 1528–1532 (2015). </span><br>
10. <span id="fn10"> S. Sakuraba, N. Matubayasi, Ermod: Fast and versatile computation software for solvation free energy with approximate theory of solutions. J. Comput. Chem. 35, 1592–1608 (2014). </span><br>

## Import of Python Modules & Auxiliary Functions

In [ ]:
#Notebook dependent libs
import numpy as np
import mdtraj as md
import parmed as pmd
import re, os

ERMODHOME = '/home/z44785r/ermod-0.3.8'
homedir = !pwd
homedir = homedir[0]
#workdir = '/home/z44785r/WORK/PEO_TMP' # FLOW
workdir = '/sqfs/home/z6b355/WORK/PEO_TMP' # Squid
print(homedir)

### Simulation settings

In [ ]:
states = { # State of simulations, (outFreq is steps per frame)
          'conf':{'Nsteps': 500000000, 'OutFreq': 1000}, # 1000 nanoseconds, 500.000 frames
          'sol': {'Nsteps':  25000000, 'OutFreq':  500}, #   50 nanoseconds,  50.000 frames
          'ref': {'Nsteps':  50000000, 'OutFreq':  500}, #  100 nanoseconds, 100.000 frames
         }

nmers = [36] # PEG polymer length
Nparticles = {       # Number of PEG and water molecules. Salt is calculated based on concentration input
    'PEG': 1,
    'Water': 10000,
}
NConfs = 100

urea_reference_concentrations = { 
   0.00: {'P0':    0, 'P1': 378, 'P2': 1472},
   2.00: {'P0':  378, 'P1':   0, 'P2': 1472},
   6.00: {'P0': 1472, 'P1':   0, 'P2':  378},
}

Tbl_concentrations = {
   0.00: {'P0':    0, 'P1': 378, 'P2': 1472},
   2.00: {'P0':  378, 'P1':   0, 'P2': 1472},
   6.00: {'P0': 1472, 'P1':   0, 'P2':  378},
}

GENERATE_ERMOD = False # Conduct ERmod analysis

## Analysis
### Prepare ERmod analysis
<img style="float: right;width:319.5px;height:312.5px;" src="Figures/ermod.png" title="ERmod analysis flow" />

The determination of solvation free energies and chemical potentials though the ERmod software is conducted in two stages as illustrated in the figure. First is the determination of the energy distribution functions from the simulation trajectories obtained from molecular dynamics using the subprogram `ermod`. Second is the solvation free energy is determined from the energy distribution functions though an approximate functional using the subprogram `slvfe`.

Documentation for the parameter file `parameters_er` which determines how the `ermod` subprogram runs can be found [here](https://sourceforge.net/p/ermod/wiki/parameters-ermod03/). In the python code below it can be noted many of the parameters has been given the keyword `[correct]`, these settings are dependent on the simulation settings and or the software being used and are thus recommended not to be changed if using the openMM setup above. The parameters given the keyword `[to be set]` are settings either collected from the output of the openMM script or set by the user, in specific the values for `maxins` and `engdiv` should be given by the user are given the default value of 1000 and 5 respectively.

Documentation for the parameter file `parameters_fe` which determines how the `slvfe` subprogram runs can be found [here](https://sourceforge.net/p/ermod/wiki/parameters-slvfe/). It is recommended not to change the parameters in this file with the exception of the temperature if conducting simulations at other temperatures than 298.15 Kelvin, and the volume. In the code below the average volume input is calculated as an average volume from the two ensemble volume averages.

In [ ]:
# ERmod settings
state_parameters = {
    'refs': {'state': 2, 'Ninserts': 200, 'Ndivisions': 50},
    'soln': {'state': 1, 'Ninserts': 200, 'Ndivisions': 50}
}

ermod_script = """
&ene_param
      slttype = {state},     ! Choose system; 1: Solution system  2,3: Reference                     [to be set]
      boxshp = 1,            ! Boxtype                                                               [correct]   
      estype = 2,            ! Ensemble; 1: NVT   2: NPT                                             [correct]   
      inptemp = 298.15,      ! Temperature (in Kelvin)                                               [correct]   
      ljformat = 5,          ! LJ form                                                               [correct]   
      cmbrule = 1,           ! Combination rule                                                      [correct]   
      ljswitch = 1,          ! Switching function for smooth LJ truncation                           [correct]   
      upljcut = 12,          ! Upper limit for LJ cutoff switching function (in Angstrom)            [correct]   
      lwljcut = 10,          ! Lower limit for LJ cutoff switching function (in Angstrom)            [correct]   
      cltype = 2,            ! Treatment of Coulomb interaction (2=PME)                              [correct]   
      elecut = 12.0,         ! Cutoff of the real-space electrostatic interaction (in angstrom)      [correct]   
      ewtoler = 0.00001,     ! Error tolerance in Ewald                                              [correct]   
      splodr = 5,            ! Order of spline function used in PME                                  [correct]   
      ms1max = {PMEnodes_x}, ! Number of meshes in PME (x)                                           [to be set] 
      ms2max = {PMEnodes_y}, ! Number of meshes in PME (y)                                           [to be set] 
      ms3max = {PMEnodes_z}, ! Number of meshes in PME (z)                                           [to be set] 
      maxins = {Ninserts},   ! Number of inserts (chosen freely, for reference ONLY)                 [to be set] 
      engdiv = {Ndivisions}, ! Number of divisions of the total simulation length (chosen freely)    [to be set] 
/
&hist
      ecprread=1,
      eclbin=5.0e-2, ecfbin=2.0e-3, ec0bin=2.0e-4, finfac=10.0e0,
      ecdmin=-160.00, ecfmns=-0.20e0, ecdcen=0.0e0, eccore=55.0e0,
      ecdmax=1.0e11, pecore=200
/
"""
    
slvfe_script = """
&fevars
clcond    = "merge",       ! Calculation type, "merge" / "basic" / "range"
numsln    = {Nsol_blocks}, ! Number of trajectory blocks in solution system
numref    = {Nref_blocks}, ! Number of trajectory blocks in reference system
numdiv    = {Nsol_blocks}, ! Number of division for statistics, usually set equal to numsln
avevolume = {avgV},        ! Average volume sampled in simulation (in Angstrom^3)

ljlrc     = 'yes',         ! Long-range correction of the Lennard-Jones interaction (avevolume must be specified)
uvread    = "yes",         ! "not" if average solute-solvent energy is calculated from engsln, instead of aveuv.tt
slfslt    = "yes",         ! "not" if the solute self energy is not read
infchk    = "yes",         ! Enable error analysis for the logarithmic-mesh part
inptemp   = 298.15,        ! Input Temperature in Kelvin
cumuint   = 'yes',         ! Enable running integral calculations.
write_mesherror = 'yes'    ! Write mesherror at the end of the slvfe output.
/
"""

EcdInfo = """species  eclbin ecfbin ec0bin finfac ecdmin ecfmns ecdcen eccore ecdmax pecore
1        5.0e-2 2.0e-3 2.0e-4 10.0e0  -30.0 -0.20e0 0.0e0   60e0 1.0e11    200
2        5.0e-2 2.0e-3 2.0e-4 10.0e0 -150.0 -0.20e0 0.0e0   20e0 1.0e11    200
3        5.0e-2 2.0e-3 2.0e-4 10.0e0  -30.0 -0.20e0 0.0e0   20e0 1.0e11    200
"""

### Generate ERmod Files
The following cell is fairly complicated and uses a mix of bash and python. While ERmod provides tools to assist in the construction of input files based on many popular molecular dynamics packages log files openMM is still yet to be supported. In the following we manually create the files. The steps are as follow:
1. Use the ERmod's `gen_structure` script to generate non-complete input scripts as well as the folders `refs` and `soln` containing pair-energy distributions at $\lambda=0$ and $\lambda=1$ respectively.
2. For soln and refs: Construct a `parameter_er` file as previously described, with updated PME parameters.
3. For soln and refs: Edit the MDinfo file containing the number of frames in the solution and reference state.
4. For soln and refs: Create a symlink named `HISTORY` linking to the joint trajectory for the solution and reference state.
5. For refs: Create a symlink named `SltConf` linking to the isolated trajectory of PEO. The conformational enemble of PEO can be controlled with the parameter `confEnsemble` taking the values `solvated` or `vacuum`.
6. Create the `parameters_fe` file with the main edit being the average volume sampled at $\lambda=1$ for long-range correction of Lennard-Jones interactions.

In [ ]:
%cd -q $homedir
if GENERATE_ERMOD:
    N_analysis = 0
    for nmer in nmers:
        nmerdir = 'PEG{}mer'.format(nmer)
        N_PEG_atoms = len(pmd.load_file('{homedir}/PDB_files/PEO-{nmer}-mer.pdb'.format(homedir=homedir, nmer=nmer)).atoms)
        
        for conc, perturb in urea_reference_concentrations.items():
            conc = '{0:.2f}'.format(conc)
            for P in ['P0', 'P1', 'P2']:
                for conf in range(NConfs):
                    %cd -q $homedir/Simulations/$nmerdir/$conc/$P/$conf
                    
                    WORKDIR = os.getcwd()
                    C = [k for k, v in Tbl_concentrations.items() if v['P0'] == perturb[P]][0]
                    C = '{0:.2f}'.format(C)
                    REFDIR = '{workdir}/Simulations/References/{conc}'.format(workdir=workdir, conc=C)
                    
                    topname = '../PEG_{nmer}_urea.parm7'.format(nmer=nmer)
                    !echo 1 | /usr/bin/python2 $ERMODHOME/tools/AMBER/gen_structure --top $topname
                    PEG = md.load_pdb('PEG_{nmer}_urea.pdb'.format(nmer=nmer), atom_indices=range(N_PEG_atoms))
                   
    
                    #########
                    # ERMOD #
                    #########
                    for ermodstate in ['soln', 'refs']:
                        %cd -q $WORKDIR/$ermodstate
            
                        with open('../run.out', 'r') as logfile:
                            lines = logfile.readlines()
                        logfile.close()
                        index = [idx for idx, s in enumerate(lines) if 'PARTICLE MESH EWALD PARAMETERS' in s][0]
                        PME_spacing = re.findall("([0-9]+[,.]+[0-9]+)", lines[index+1])
                        N_gridpoints = (*re.findall("([0-9]+)", lines[index+2]),
                                        *re.findall("([0-9]+)", lines[index+3]),
                                        *re.findall("([0-9]+)", lines[index+4]))
            
                        parameters = state_parameters[ermodstate]
                        with open('parameters_er', 'w') as f:
                            f.write(ermod_script.format(state=parameters['state'], Ninserts=parameters['Ninserts'], Ndivisions=parameters['Ndivisions'],
                                                        PMEnodes_x=N_gridpoints[0], PMEnodes_y=N_gridpoints[1], PMEnodes_z=N_gridpoints[2]))
            
                        with open('MDinfo', 'r+') as f:
                            lines = f.readlines()
                            num_frames =  int(states[ermodstate[:3]]['Nsteps']/states[ermodstate[:3]]['OutFreq'])
                            lines[0] = lines[0].replace('FRAMES', str(num_frames))
                            f.seek(0)
                            f.truncate()
                            for line in lines:
                                f.write(line)
                        f.close()
                        
                        with open('EcdInfo', 'w') as f:
                            f.write(EcdInfo)
                        f.close()
                    
                        with open('SltAtomGroup', 'w') as f:
                            for residue in PEG.topology.residues:
                                indexes = []
                                for atom in residue.atoms:
                                    indexes.append(atom.index+1)
                                f.write(" ".join(map(str, indexes))+'\n')
                        f.close()
                    
                        if ermodstate == 'soln':
                            simpath = WORKDIR.split(sep='PEO_TMP')[1]
                            simpath = workdir+simpath
                            !ln -s $simpath/trajectory.xtc HISTORY
            
                        if ermodstate == 'refs':
                            !ln -s $REFDIR/trajectory.xtc HISTORY
                            PEG.save('structure.pdb')
                            !sed -e 1,3d structure.pdb | awk '{print $7 "\t" $8 "\t" $9}' > structure
                            !paste SltInfo structure > SltInfo_new
                            !mv SltInfo_new SltInfo
                
                    #########
                    # SLVFE #
                    #########
                    %cd -q $WORKDIR
                    dirc = '{homedir}/Simulations/References/{conc}/output.dat'.format(homedir=homedir,conc=C)
                    vol_data = np.loadtxt(dirc, usecols=4, skiprows=1)
                    avgVolume = vol_data.mean()*1000
                    with open('parameters_fe', 'w') as f:
                        f.write(slvfe_script.format(Nsol_blocks=state_parameters['soln']['Ndivisions'],
                                                Nref_blocks=state_parameters['refs']['Ndivisions'],
                                                avgV=avgVolume))
                    print('Wrote ERmod analysis files to '+os.getcwd())
                    N_analysis+=1

### Submit ERmod analysis
Submit script for servers employing job scheduling. The below example is utilizing PBS (for a quick guide see [here](https://latisresearch.umn.edu/creating-a-PBS-script)). However the code below may be edited to utilize Slurm instead (documentation [here](https://slurm.schedmd.com)) by changing the variable `submit_script` and by executing the commands `!sbatch submit_ermod.pbs` instead of `qsub`.

In [ ]:
%cd -q $homedir

submit_script_flow="""#!/bin/bash
## FLOW
#PJM -L rscunit=fx
#PJM -L rscgrp=fx-small
#PJM -L node=1
#PJM --mpi proc=30
#PJM -L elapse=168:00:00
#PJM -o ermod.out
#PJM -e ermod.err

source ~/.bashrc
source ~/.bash_profile

export ERMOD=/home/z44785r/type_1/ermod-0.3.8
export ERMOD_PLUGINS=/home/z44785r/type_1/ermod-0.3.8/vmdplugins/libexec

PATH=$PATH:/home/z44785r/type_1/ermod-0.3.8
export PATH

module load fftw/3.3.8

sleep 13

cd {path}

# Calculate solvation energy
# Solution step
cd soln
mpiexec /home/z44785r/type_1/ermod-0.3.8/ermod
cd ..

# Reference step
cd refs
mpiexec /home/z44785r/type_1/ermod-0.3.8/ermod
cd ..

/home/z44785r/type_1/ermod-0.3.8/slvfe
"""

submit_script_squid="""#!/bin/bash
## SQUID
#PBS -q SQUID
#PBS --group=hp230109
#PBS -b 1
#PBS -l cpunum_job=76
#PBS -l elapstim_req=120:00:00
#PBS -T intmpi
#PBS -v OMP_NUM_THREADS=1
#PBS -o ermod.out
#PBS -e ermod.err
source ~/.bashrc
source ~/.bash_profile

export ERMOD=/sqfs/home/z6b355/ermod-0.3.8
export ERMOD_PLUGINS=/sqfs/home/z6b355/ermod-0.3.8/vmdplugins/libexec

PATH=$PATH:/sqfs/home/z6b355/ermod-0.3.8
export PATH

sleep 13

cd {path}

# Calculate solvation energy
# Solution step
cd soln
mpiexec.hydra ${{NQSV_MPIOPTS}} -n 76 ermod
cd ..

# Reference step
cd refs
mpiexec.hydra ${{NQSV_MPIOPTS}} -n 76 ermod
cd ..

slvfe
"""

master_submit_script="""#!/bin/bash
read -p "Will submit {N_analysis} ERmod analysis. Do you want to proceed? (yes/no) " yn

case $yn in 
    yes ) echo submitting...;;
    no ) echo exiting...;
         exit;;
    * ) echo invalid response;
        exit 1;;
esac

"""
N_analysis=300
if GENERATE_ERMOD:
    with open('Simulations/master_ERmod.sh', 'w') as ff:
        ff.write(master_submit_script.format(N_analysis=N_analysis))
    
        for nmer in nmers:
            nmerdir = 'PEG{}mer'.format(nmer)
            for conc in urea_reference_concentrations:
                conc = '{0:.2f}'.format(conc)
                for P in ['P0', 'P1', 'P2']:
                    for conf in range(NConfs):
                        %cd -q $homedir/Simulations/$nmerdir/$conc/$P/$conf
                        
                        simpath = os.getcwd()
                        simpath = simpath.split(sep='PEO_TMP')[1]
                        simpath = workdir+simpath
                        with open('submit_ermod.pbs', 'w') as f:
                            f.write(submit_script_squid.format(path=simpath))
                        f.close()
                        #s = 'cd {}\npjsub submit_ermod.pbs\n\n'.format(simpath) # FLOW
                        s = 'cd {}\nqsub submit_ermod.pbs\n\n'.format(simpath) # Squid
                        ff.write(s)
    ff.close()

## Obtaining solvation free energies from ermod output

In [ ]:
%cd -q $homedir

slvfe       = np.zeros(shape=(len(urea_reference_concentrations), len(urea_reference_concentrations), NConfs))
slvfe_err   = np.zeros(shape=(len(urea_reference_concentrations), len(urea_reference_concentrations), NConfs))
mesh_err    = np.zeros(shape=(len(urea_reference_concentrations), len(urea_reference_concentrations), NConfs))
self_energy = np.zeros(shape=(len(urea_reference_concentrations), len(urea_reference_concentrations), NConfs))

for a in [slvfe,slvfe_err,mesh_err,self_energy]:
    a[:] = np.nan

for i, (conc_ensemble, perturb1) in enumerate(urea_reference_concentrations.items()):
    conc_ensemble = '{0:.2f}'.format(conc_ensemble)
    for j, (conc_perturbation, perturb2) in enumerate(urea_reference_concentrations.items()):
        conc_perturbation = '{0:.2f}'.format(conc_perturbation)
        P = str(*[k for k, v in perturb1.items() if v==perturb2['P0']])
        if P == '':
            continue
        
        for conf in range(NConfs):
            path = 'Simulations/PEG36mer/{conc}/{P}/{conf}/'.format(conc=conc_ensemble, P=P, conf=conf)
            with open(path+'ermod.out','r') as f:
                lines = f.readlines()
            f.close()
            
            if perturb1[P] == 0:
                mod = 0
            else:
                mod = 1

            for lineNR, line in enumerate(lines):
                if "cumulative average & 95% error for solvation free energy" in line:
                    slvfe[i,j,conf] = float(lines[lineNR+50+mod].split()[1])
                    slvfe_err[i,j,conf] = float(lines[lineNR+50+mod].split()[2])
                elif "Mesh error is" in line:
                    mesh_err[i,j,conf] = float(line.split()[3])
                elif "Self-energy" in line:
                    self_energy[i,j,conf] = float(line.split()[5])


# Save the results to the desk in binary numpy format
np.save('Data/slvfe_PEG_Urea.npy',     slvfe)
np.save('Data/slvfe_err_PEG_Urea.npy', slvfe_err)
np.save('Data/mesh_err_PEG_Urea.npy',  mesh_err)
np.save('Data/self_err_PEG_Urea.npy',  self_energy)